In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import requests
import json

In [2]:
url = 'https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=2500'

In [3]:
for each in range(30):
    url = 'https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after={}d'.format(each)
    print(url)

https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=0d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=1d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=2d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=3d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=4d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=5d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=6d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=7d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=8d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=9d
https://api.pushshift.io/reddit/search/submission/?subreddit=finance&size=500&after=10d
https://api.pushshift.io/reddit/search/sub

In [8]:
###!pip install progressbar2



In [9]:
from progressbar import ProgressBar

In [10]:
def query_pushshift(subreddit,kind ='submission',skip = 2, times =25, 
                    subfield = ['title','selftext', 'subreddit',
                                'created_utc', 'author', 'id',
                                'num_comments', 'score','full_link', 'is_self']):
    

    
    bar = ProgressBar()
    mylist = []
    for each in bar(range(1,times)):
        url = 'https://api.pushshift.io/reddit/search/{}/?subreddit={}&size=500&after={}d'.format(kind,subreddit,str(each*skip))
        response = requests.get(url)
        assert response.status_code == 200
        data = response.json()['data']
        df = pd.DataFrame.from_dict(data)
        mylist.append(df)
        time.sleep(2)
      
    full = pd.concat(mylist,)
    
    if kind == 'submission':
        
        full = full[subfield]
        full.drop_duplicates()
        full = full.loc[full['is_self'] == True]

    def get_date(created):
        return dt.date.fromtimestamp(created)
    
    _timestamp = full['created_utc'].apply(get_date)
    
    full['timestamp'] = _timestamp
    full.reset_index
    
    print(full.shape)
    
    return full

In [11]:
df=query_pushshift("Changemyview",skip = 3 , times = 50)

100% (49 of 49) |########################| Elapsed Time: 0:03:52 Time:  0:03:52
C:\Users\tbaca\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



(24168, 11)


In [ ]:
pd.set_option('display.max_colwidth', -1)

In [14]:
df.to_csv(path_or_buf="Changemyview_Final_scrape.csv", encoding = "utf-8")



PiyasaTL; Dolar kuru, Döviz kurları, Piyasa bilgileri, Altın fiyatları ve Borsa verileri.                                                                               112
Commercial Domestic Investigations                                                                                                                                       97
Commercial Domestic Investigations Review                                                                                                                                42
personal loan Delhi NCR                                                                                                                                                  36
Personal Loan facility in delhi ncr                                                                                                                                      35
Personal Loan in Delhi NCR                                                                                                                  

In [15]:
r_finance = pd.read_csv("./Finance_Final_Scrape.csv")
r_changemyview = pd.read_csv("./Changemyview_Final_scrape.csv")

In [ ]:
from sklearn import svm, linear_model, datasets
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()

params = {'n_neighbors': [2,3,5,7,9,11],
         'metric': ['minkowski', 'euclidean', 'manhattan']}

grid = GridSearchCV(knn,params)
grid.fit(df,y.ravel())

grid.best_params_

grid.best_score_